## Using Llama

In [ ]:
import os
import sqlite3
import logging
from llama_cpp import Llama

# -------------------
# Logging setup
# -------------------
logging.basicConfig(
    level=logging.DEBUG,
    format='[%(asctime)s] %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# -------------------
# Path configuration
# -------------------
BASE_DIR = os.getcwd()
DB_PATH = os.path.join(BASE_DIR,  "..\data\example_data.db")
MODEL_PATH = os.path.join(BASE_DIR, "..\models\mistral-7b-instruct-v0.1.Q4_K_M.gguf")
SCHEMA_PATH = os.path.join(BASE_DIR, "..\schema_description.txt")

# -------------------
# Load LLaMA-compatible model (Mistral)
# -------------------
_llm_instance = None
def load_llm():
    global _llm_instance
    if _llm_instance is None:
        logging.info("Loading LLM model from: %s", MODEL_PATH)
        _llm_instance = Llama(
            model_path=MODEL_PATH,
            n_ctx=1024,
            n_threads=18,  # adjust depending on your CPU,
            n_batch = 128,
            n_gpu_layers = 16
        )
        logging.info("LLM model loaded successfully.")
    return _llm_instance

# -------------------
# Generate SQL query from prompt + schema
# -------------------
def generate_sql_from_prompt(prompt: str, schema_path: str = SCHEMA_PATH) -> str:
    logging.debug("Reading schema from: %s", schema_path)
    
    with open(schema_path, "r") as f:
        schema_text = f.read()

    llm = load_llm()

    
    full_prompt = (
        f"You are a helpful assistant that writes SQL queries.\n\n"
        f"Given the following database schema:\n\n"
        f"{schema_text}\n\n"
        f"Write a valid SQL query to answer the following question:\n"
        f"{prompt}\n\n"
        f"Make sure the SQL is complete and executable."
        f"SQL:"
    )

    #number of tokens calculation
    tokens = llm.tokenize(full_prompt.encode("utf-8"))
    num_tokens = len(tokens)
    logging.debug(f"Total tokens: {num_tokens}")


    logging.debug("Sending prompt to LLM:\n%s", full_prompt)

    output = llm(
        prompt=full_prompt,
        max_tokens=80,
        stop=["#", ";"]
    )

    sql = output["choices"][0]["text"].strip()
    #logging.info("Generated SQL:\n%s", sql)

    # Remove Markdown-style code block if present
    if sql.startswith("```"):
        sql = sql.strip("`")             # remove backticks
        lines = sql.splitlines()
        if lines[0].lower().startswith("sql"):
            lines = lines[1:]            # remove ```sql
        sql = "\n".join(lines).strip()   # re-join remaining lines
        return sql

# -------------------
# Execute SQL over SQLite DB
# -------------------
def run_sql_query(sql: str):
    logging.debug("Running SQL query:\n%s", sql)
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute(sql)
        rows = cursor.fetchall()
        col_names = [description[0] for description in cursor.description]
        conn.close()
        logging.info("Query executed successfully. Rows returned: %d", len(rows))
        return col_names, rows
    except sqlite3.Error as e:
        logging.error("SQLite error: %s", e)
        return None, f"SQLite error: {e}"


In [40]:
prompt = "Show the total sales amount per year for each product."  #initial test
sql = generate_sql_from_prompt(prompt)
print("Generated SQL:\n", sql)

columns, results = run_sql_query(sql)
print("Results:\n")
if isinstance(results, list):
    for row in results:
        print(dict(zip(columns, row)))
else:
    print(results)

[2025-06-06 15:10:43] DEBUG - Reading schema from: c:\Users\usuario\Desktop\Personal\Data science\LLM RAG\app_flask_v2\test\..\schema_description.txt
[2025-06-06 15:10:43] INFO - Loading LLM model from: c:\Users\usuario\Desktop\Personal\Data science\LLM RAG\app_flask_v2\test\..\models\mistral-7b-instruct-v0.1.Q4_K_M.gguf
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from c:\Users\usuario\Desktop\Personal\Data science\LLM RAG\app_flask_v2\test\..\models\mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:       

Generated SQL:
 SELECT Product, Year, SUM(Sales_Amount) AS Total_Sales
FROM sales
GROUP BY Product, Year
ORDER BY Product, Year
